In [ ]:
#@title import efficientvim
import torch
import torch.nn as nn
from timm.layers import SqueezeExcite


class LayerNorm2D(nn.Module):
    """LayerNorm for channels of 2D tensor(B C H W)"""
    def __init__(self, num_channels, eps=1e-5, affine=True):
        super(LayerNorm2D, self).__init__()
        self.num_channels = num_channels
        self.eps = eps
        self.affine = affine

        if self.affine:
            self.weight = nn.Parameter(torch.ones(1, num_channels, 1, 1))
            self.bias = nn.Parameter(torch.zeros(1, num_channels, 1, 1))
        else:
            self.register_parameter('weight', None)
            self.register_parameter('bias', None)

    def forward(self, x):
        mean = x.mean(dim=1, keepdim=True)  # (B, 1, H, W)
        var = x.var(dim=1, keepdim=True, unbiased=False)  # (B, 1, H, W)

        x_normalized = (x - mean) / torch.sqrt(var + self.eps)  # (B, C, H, W)

        if self.affine:
            x_normalized = x_normalized * self.weight + self.bias

        return x_normalized


class LayerNorm1D(nn.Module):
    """LayerNorm for channels of 1D tensor(B C L)"""
    def __init__(self, num_channels, eps=1e-5, affine=True):
        super(LayerNorm1D, self).__init__()
        self.num_channels = num_channels
        self.eps = eps
        self.affine = affine

        if self.affine:
            self.weight = nn.Parameter(torch.ones(1, num_channels, 1))
            self.bias = nn.Parameter(torch.zeros(1, num_channels, 1))
        else:
            self.register_parameter('weight', None)
            self.register_parameter('bias', None)

    def forward(self, x):
        mean = x.mean(dim=1, keepdim=True)  # (B, 1, H, W)
        var = x.var(dim=1, keepdim=True, unbiased=False)  # (B, 1, H, W)

        x_normalized = (x - mean) / torch.sqrt(var + self.eps)  # (B, C, H, W)

        if self.affine:
            x_normalized = x_normalized * self.weight + self.bias

        return x_normalized


class ConvLayer2D(nn.Module):
    def __init__(self, in_dim, out_dim, kernel_size=3, stride=1, padding=0, dilation=1, groups=1, norm=nn.BatchNorm2d, act_layer=nn.ReLU, bn_weight_init=1):
        super(ConvLayer2D, self).__init__()
        self.conv = nn.Conv2d(
            in_dim,
            out_dim,
            kernel_size=(kernel_size, kernel_size),
            stride=(stride, stride),
            padding=(padding, padding),
            dilation=(dilation, dilation),
            groups=groups,
            bias=False
        )
        self.norm = norm(num_features=out_dim) if norm else None
        self.act = act_layer() if act_layer else None

        if self.norm:
            torch.nn.init.constant_(self.norm.weight, bn_weight_init)
            torch.nn.init.constant_(self.norm.bias, 0)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.conv(x)
        if self.norm:
            x = self.norm(x)
        if self.act:
            x = self.act(x)
        return x


class ConvLayer1D(nn.Module):
    def __init__(self, in_dim, out_dim, kernel_size=3, stride=1, padding=0, dilation=1, groups=1, norm=nn.BatchNorm1d, act_layer=nn.ReLU, bn_weight_init=1):
        super(ConvLayer1D, self).__init__()
        self.conv = nn.Conv1d(
            in_dim,
            out_dim,
            kernel_size=kernel_size,
            stride=stride,
            padding=padding,
            dilation=dilation,
            groups=groups,
            bias=False
        )
        self.norm = norm(num_features=out_dim) if norm else None
        self.act = act_layer() if act_layer else None

        if self.norm:
            torch.nn.init.constant_(self.norm.weight, bn_weight_init)
            torch.nn.init.constant_(self.norm.bias, 0)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.conv(x)
        if self.norm:
            x = self.norm(x)
        if self.act:
            x = self.act(x)
        return x


class FFN(nn.Module):
    def __init__(self, in_dim, dim):
        super().__init__()
        self.fc1 = ConvLayer2D(in_dim, dim, 1)
        self.fc2 = ConvLayer2D(dim, in_dim, 1, act_layer=None, bn_weight_init=0)

    def forward(self, x):
        x = self.fc2(self.fc1(x))
        return x


class Stem(nn.Module):
    def __init__(self,  in_dim=3, dim=96):
        super().__init__()
        self.conv = nn.Sequential(
            ConvLayer2D(in_dim, dim // 8, kernel_size=3, stride=2, padding=1),
            ConvLayer2D(dim // 8, dim // 4, kernel_size=3, stride=2, padding=1),
            ConvLayer2D(dim // 4, dim // 2, kernel_size=3, stride=2, padding=1),
            ConvLayer2D(dim // 2, dim, kernel_size=3, stride=2, padding=1, act_layer=None))

    def forward(self, x):
        x = self.conv(x)
        return x

class PatchMerging(nn.Module):
    def __init__(self,  in_dim, out_dim, ratio=4.0):
        super().__init__()
        hidden_dim = int(out_dim * ratio)
        self.conv = nn.Sequential(
            ConvLayer2D(in_dim, hidden_dim, kernel_size=1),
            ConvLayer2D(hidden_dim, hidden_dim, kernel_size=3, stride=2, padding=1, groups=hidden_dim),
            SqueezeExcite(hidden_dim, .25),
            ConvLayer2D(hidden_dim, out_dim, kernel_size=1, act_layer=None)
        )

        self.dwconv1 = ConvLayer2D(in_dim, in_dim, 3, padding=1, groups=in_dim, act_layer=None)
        self.dwconv2 = ConvLayer2D(out_dim, out_dim, 3, padding=1, groups=out_dim, act_layer=None)

    def forward(self, x):
        x = x + self.dwconv1(x)
        x = self.conv(x)
        x = x + self.dwconv2(x)
        return x

!pip install fvcore

import math

import torch
import torch.nn as nn
import torch.nn.functional as F

from timm.layers import trunc_normal_
from timm.models import register_model
from fvcore.nn import flop_count

class HSMSSD(nn.Module):
    def __init__(self, d_model, ssd_expand=1, A_init_range=(1, 16), state_dim = 64):
        super().__init__()
        self.ssd_expand = ssd_expand
        self.d_inner = int(self.ssd_expand * d_model)
        self.state_dim = state_dim

        self.BCdt_proj = ConvLayer1D(d_model, 3*state_dim, 1, norm=None, act_layer=None)
        conv_dim = self.state_dim*3
        self.dw = ConvLayer2D(conv_dim, conv_dim, 3,1,1, groups=conv_dim, norm=None, act_layer=None, bn_weight_init=0)
        self.hz_proj = ConvLayer1D(d_model, 2*self.d_inner, 1, norm=None, act_layer=None)
        self.out_proj = ConvLayer1D(self.d_inner, d_model, 1, norm=None, act_layer=None, bn_weight_init=0)

        A = torch.empty(self.state_dim, dtype=torch.float32).uniform_(*A_init_range)
        self.A = torch.nn.Parameter(A)
        self.act = nn.SiLU()
        self.D = nn.Parameter(torch.ones(1))
        self.D._no_weight_decay = True

    def forward(self, x):
        batch, _, L= x.shape
        H = int(math.sqrt(L))

        BCdt = self.dw(self.BCdt_proj(x).view(batch,-1, H, H)).flatten(2)
        B,C,dt = torch.split(BCdt, [self.state_dim, self.state_dim,  self.state_dim], dim=1)
        A = (dt + self.A.view(1,-1,1)).softmax(-1)

        AB = (A * B)
        h = x @ AB.transpose(-2,-1)

        h, z = torch.split(self.hz_proj(h), [self.d_inner, self.d_inner], dim=1)
        h = self.out_proj(h * self.act(z)+ h * self.D)
        y = h @ C # B C N, B C L -> B C L

        y = y.view(batch,-1,H,H).contiguous()# + x * self.D  # B C H W
        return y, h


class EfficientViMBlock(nn.Module):
    def __init__(self, dim, mlp_ratio=4., ssd_expand=1, state_dim=64):
        super().__init__()
        self.dim = dim
        self.mlp_ratio = mlp_ratio

        self.mixer = HSMSSD(d_model=dim, ssd_expand=ssd_expand,state_dim=state_dim)
        self.norm = LayerNorm1D(dim)

        self.dwconv1 = ConvLayer2D(dim, dim, 3, padding=1, groups=dim, bn_weight_init=0, act_layer = None)
        self.dwconv2 = ConvLayer2D(dim, dim, 3, padding=1, groups=dim, bn_weight_init=0, act_layer = None)

        self.ffn = FFN(in_dim=dim, dim=int(dim * mlp_ratio))

        #LayerScale
        self.alpha = nn.Parameter(1e-4 * torch.ones(4,dim), requires_grad=True)

    def forward(self, x):
        alpha = torch.sigmoid(self.alpha).view(4,-1,1,1)

        # DWconv1
        x = (1-alpha[0]) * x + alpha[0] * self.dwconv1(x)

        # HSM-SSD
        x_prev = x
        x, h = self.mixer(self.norm(x.flatten(2)))
        x = (1-alpha[1]) * x_prev + alpha[1] * x

        # DWConv2
        x = (1-alpha[2]) * x + alpha[2] * self.dwconv2(x)

        # FFN
        x = (1-alpha[3]) * x + alpha[3] * self.ffn(x)
        return x, h


class EfficientViMStage(nn.Module):
    def __init__(self, in_dim, out_dim, depth,  mlp_ratio=4.,downsample=None, ssd_expand=1, state_dim=64):
        super().__init__()
        self.depth = depth
        self.blocks = nn.ModuleList([
            EfficientViMBlock(dim=in_dim, mlp_ratio=mlp_ratio, ssd_expand=ssd_expand, state_dim=state_dim) for _ in range(depth)])

        self.downsample = downsample(in_dim=in_dim, out_dim =out_dim) if downsample is not None else None

    def forward(self, x):
        for blk in self.blocks:
            x, h = blk(x)

        x_out = x
        if self.downsample is not None:
            x = self.downsample(x)
        return x, x_out, h


class EfficientViM(nn.Module):
    def __init__(self, in_dim=3, num_classes=1, embed_dim=[128,256,512], depths=[2, 2, 2], mlp_ratio=4., ssd_expand=1, state_dim=[49,25,9], distillation=False, **kwargs):
        super().__init__()
        self.num_layers = len(depths)
        self.num_classes = num_classes
        self.distillation =distillation
        self.patch_embed = Stem(in_dim=in_dim, dim=embed_dim[0])
        PatchMergingBlock = PatchMerging

        # build stages
        self.stages = nn.ModuleList()
        for i_layer in range(self.num_layers):
            stage = EfficientViMStage(in_dim=int(embed_dim[i_layer]),
                               out_dim=int(embed_dim[i_layer+1]) if (i_layer < self.num_layers - 1) else None,
                               depth=depths[i_layer],
                               mlp_ratio=mlp_ratio,
                               downsample=PatchMergingBlock if (i_layer < self.num_layers - 1) else None,
                               ssd_expand=ssd_expand,
                               state_dim = state_dim[i_layer])
            self.stages.append(stage)

        # Weights for multi-stage hidden-state Fusion
        self.weights = nn.Parameter(torch.ones(4))
        self.norm = nn.ModuleList([
            LayerNorm1D(embed_dim[0]),
            LayerNorm1D(embed_dim[1]),
            LayerNorm1D(embed_dim[2]),
            LayerNorm2D(embed_dim[2]),
        ])
        self.heads = nn.ModuleList([
            nn.Linear(embed_dim[0], num_classes) if num_classes > 0 else nn.Identity(),
            nn.Linear(embed_dim[1], num_classes) if num_classes > 0 else nn.Identity(),
            nn.Linear(embed_dim[2], num_classes) if num_classes > 0 else nn.Identity(),
            nn.Linear(embed_dim[2], num_classes) if num_classes > 0 else nn.Identity()
        ])

        if distillation:
            self.weights_dist = nn.Parameter(torch.ones(4))
            self.heads_dist = nn.ModuleList([
                nn.Linear(embed_dim[0], num_classes) if num_classes > 0 else nn.Identity(),
                nn.Linear(embed_dim[1], num_classes) if num_classes > 0 else nn.Identity(),
                nn.Linear(embed_dim[2], num_classes) if num_classes > 0 else nn.Identity(),
                nn.Linear(embed_dim[2], num_classes) if num_classes > 0 else nn.Identity()
            ])

        self.apply(self._init_weights)

    def _init_weights(self, m):
        if isinstance(m, nn.Linear):
            trunc_normal_(m.weight, std=.02)
            if isinstance(m, nn.Linear) and m.bias is not None:
                nn.init.constant_(m.bias, 0)
        elif isinstance(m, LayerNorm2D):
            nn.init.constant_(m.bias, 0)
            nn.init.constant_(m.weight, 1.0)
        elif isinstance(m, LayerNorm1D):
            nn.init.constant_(m.bias, 0)
            nn.init.constant_(m.weight, 1.0)
        elif isinstance(m, nn.BatchNorm2d):
            nn.init.constant_(m.bias, 0)
            nn.init.constant_(m.weight, 1.0)
        elif isinstance(m, nn.BatchNorm1d):
            nn.init.constant_(m.bias, 0)
            nn.init.constant_(m.weight, 1.0)

    @torch.no_grad()
    def flops(self, shape=(3, 224, 224)):
        supported_ops = {
            "aten::silu": None,
            "aten::neg": None,
            "aten::exp": None,
            "aten::flip": None,
            "aten::softmax": None,
            "aten::sigmoid": None,
            "aten::mul": None,
            "aten::add": None,
            "aten::mean": None,
            "aten::var": None,
            "aten::sub": None,
            "aten::sqrt": None,
            "aten::div": None,
            "aten::rsub": None,
            "aten::adaptive_avg_pool1d": None,
        }
        import copy
        model = copy.deepcopy(self)
        model.cuda().eval()

        input = torch.randn((1, *shape), device=next(model.parameters()).device)
        Gflops, unsupported = flop_count(model=model, inputs=(input,), supported_ops=supported_ops)

        del model, input

        return sum(Gflops.values()) * 1e9

    def forward(self, x):
        x = self.patch_embed(x)

        weights = self.weights.softmax(-1)
        z = torch.zeros((x.shape[0], self.num_classes), device=x.device)
        if self.distillation:
            weights_dist = self.weights_dist.softmax(-1)
            z_dist = torch.zeros((x.shape[0], self.num_classes), device=x.device)

        for i, stage in enumerate(self.stages):
            x, x_out, h = stage(x)

            h = self.norm[i](h)
            h = torch.nn.functional.adaptive_avg_pool1d(h, 1).flatten(1)
            z = z + weights[i] * self.heads[i](h)
            if self.distillation:
                z_dist = z_dist + weights_dist[i] * self.heads_dist[i](h)

        x = self.norm[3](x)
        x = torch.nn.functional.adaptive_avg_pool2d(x, 1).flatten(1)
        z = z + weights[3] * self.heads[3](x)

        if self.distillation:
            z_dist = z_dist + weights_dist[3] * self.heads_dist[3](x)
            z= z, z_dist
            if not self.training:
                z = (z[0] + z[1]) / 2

        return z


@register_model
def EfficientViM_M1(pretrained=False, **kwargs):
    model = EfficientViM(
        in_dim=3,
        embed_dim=[128,192,320],
        depths=[2,2,2],
        mlp_ratio=4.,
        ssd_expand=1.,
        state_dim=[49,25,9],
        **kwargs)
    return model


@register_model
def EfficientViM_M2(pretrained=False, **kwargs):
    model = EfficientViM(
        in_dim=3,
        embed_dim=[128,256,512],
        depths=[2,2,2],
        mlp_ratio=4.,
        ssd_expand=1.,
        state_dim=[49,25,9],
        **kwargs)
    return model


@register_model
def EfficientViM_M3(pretrained=False, **kwargs):
    model = EfficientViM(
        in_dim=3,
        embed_dim=[224,320,512],
        depths=[2,2,2],
        mlp_ratio=4.,
        ssd_expand=1.,
        state_dim=[49,25,9],
        **kwargs)
    return model


@register_model
def EfficientViM_M4(pretrained=False, **kwargs):
    model = EfficientViM(
        in_dim=3,
        embed_dim=[224,320,512],
        depths=[3,4,2],
        mlp_ratio=4.,
        ssd_expand=1.,
        state_dim=[64,32,16],
        **kwargs)
    return model

In [ ]:
# Make modified ViMamba
new_vim_pneumnist=EfficientViM_M1(pretrained=True)

In [ ]:
#@title Import in the ViT
!pip install einops

from torch import nn
from einops.layers.torch import Rearrange
from torch import Tensor


class PatchEmbedding(nn.Module):
    def __init__(self, in_channels = 3, patch_size = 8, emb_size = 128):
        self.patch_size = patch_size
        super().__init__()
        self.projection = nn.Sequential(
            # break-down the image in s1 x s2 patches and flat them
            Rearrange('b c (h p1) (w p2) -> b (h w) (p1 p2 c)', p1=patch_size, p2=patch_size),
            nn.Linear(patch_size * patch_size * in_channels, emb_size)
        )

    def forward(self, x: Tensor) -> Tensor:
        x = self.projection(x)
        return x

from einops import rearrange

class Attention(nn.Module):
    def __init__(self, dim, n_heads, dropout):
        super().__init__()
        self.n_heads = n_heads
        self.att = torch.nn.MultiheadAttention(embed_dim=dim,
                                               num_heads=n_heads,
                                               dropout=dropout)
        self.q = torch.nn.Linear(dim, dim)
        self.k = torch.nn.Linear(dim, dim)
        self.v = torch.nn.Linear(dim, dim)

    def forward(self, x):
        q = self.q(x)
        k = self.k(x)
        v = self.v(x)
        attn_output, attn_output_weights = self.att(x, x, x)
        return attn_output

class PreNorm(nn.Module):
    def __init__(self, dim, fn):
        super().__init__()
        self.norm = nn.LayerNorm(dim)
        self.fn = fn
    def forward(self, x, **kwargs):
        return self.fn(self.norm(x), **kwargs)

class FeedForward(nn.Sequential):
    def __init__(self, dim, hidden_dim, dropout = 0.):
        super().__init__(
            nn.Linear(dim, hidden_dim),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim, dim),
            nn.Dropout(dropout)
        )

class ResidualAdd(nn.Module):
    def __init__(self, fn):
        super().__init__()
        self.fn = fn

    def forward(self, x, **kwargs):
        res = x
        x = self.fn(x, **kwargs)
        x += res
        return x

from einops import repeat

class ViT(nn.Module):
    def __init__(self, ch=3, img_size=224, patch_size=16, emb_dim=32,
                n_layers=6, out_dim=1, dropout=0.1, heads=2):
        super(ViT, self).__init__()

        # Attributes
        self.channels = ch
        self.height = img_size
        self.width = img_size
        self.patch_size = patch_size
        self.n_layers = n_layers

        # Patching
        self.patch_embedding = PatchEmbedding(in_channels=ch,
                                              patch_size=patch_size,
                                              emb_size=emb_dim)
        # Learnable params
        num_patches = (img_size // patch_size) ** 2
        self.pos_embedding = nn.Parameter(
            torch.randn(1, num_patches + 1, emb_dim))
        self.cls_token = nn.Parameter(torch.rand(1, 1, emb_dim))

        # Transformer Encoder
        self.layers = nn.ModuleList([])
        for _ in range(n_layers):
            transformer_block = nn.Sequential(
                ResidualAdd(PreNorm(emb_dim, Attention(emb_dim, n_heads = heads, dropout = dropout))),
                ResidualAdd(PreNorm(emb_dim, FeedForward(emb_dim, emb_dim, dropout = dropout))))
            self.layers.append(transformer_block)

        # Classification head
        self.head = nn.Sequential(nn.LayerNorm(emb_dim), nn.Linear(emb_dim, out_dim))


    def forward(self, img):
        # Get patch embedding vectors
        x = self.patch_embedding(img)
        b, n, _ = x.shape

        # Add cls token to inputs
        cls_tokens = repeat(self.cls_token, '1 1 d -> b 1 d', b = b)
        x = torch.cat([cls_tokens, x], dim=1)
        x += self.pos_embedding[:, :(n + 1)]

        # Transformer layers
        for i in range(self.n_layers):
            x = self.layers[i](x)

        # Output based on classification token
        return self.head(x[:, 0, :])

In [ ]:
!pip install vit-pytorch

import torch
from vit_pytorch import SimpleViT

new_vit_pneumnist = SimpleViT(
    image_size = 224,
    patch_size = 32,
    num_classes = 1,
    dim = 1024,
    depth = 6,
    heads = 16,
    mlp_dim = 2048
)

In [ ]:
# Import needed libraries, torch for nn, numpy for numerical stuff, and plt for quick graphs.

import torch
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Create a function for clearning gpu memory
import gc

def clear_gpu():
  gc.collect()
  torch.cuda.empty_cache()

In [ ]:
# Help with memory usage.
!export PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True

## Data Prep

In [ ]:
!pip install medmnist

In [ ]:
# Import PneuMNIST, which we will be training on.

from medmnist import PneumoniaMNIST

In [ ]:
# Import needed nn and data tools that just streamline the flow of things.

import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import torchvision.transforms as transforms

In [ ]:
# Define our tensor transforms to our PneuMNIST dataset.

download_transform=transforms.Compose([
    # Create 3 channels from one just in case
    transforms.Grayscale(num_output_channels=3),
    transforms.ToTensor(),
    transforms.Normalize(mean=[.5], std=[.5])
])

In [ ]:
# Download Train and Test PneuMNSIT, this is around 224 px which is good for the ViT

pneu_mnist_train=PneumoniaMNIST(split='train', download=True, size=224, transform=download_transform)
print('train download done!')
pneu_mnist_test=PneumoniaMNIST(split='test', download=True, size=224, transform=download_transform)
print('test download done!')

Using downloaded and verified file: /root/.medmnist/pneumoniamnist_224.npz
train download done!
Using downloaded and verified file: /root/.medmnist/pneumoniamnist_224.npz
test download done!


In [ ]:
# Create the test and train datasets, REMEMBER BATCH SIZE!

test_pneumnist_dataloader=data.DataLoader(dataset=pneu_mnist_test, batch_size=32, num_workers=1, shuffle=False)
train_pneumnist=data.DataLoader(dataset=pneu_mnist_train, batch_size=32, num_workers=1, shuffle=True)

In [ ]:
# Delete old torch datasets now that dataloaders are created.

del pneu_mnist_train
del pneu_mnist_test

## DENOISING CNN

In [ ]:
# Setting up our device to train the nn on.
device=torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [ ]:
# Creating our denoising CNN.

class DenoiseCNN(nn.Module):
  def __init__(self):
    super(DenoiseCNN, self).__init__()
    # in layer
    self.conv1 = nn.Conv2d(in_channels=3, out_channels=64, kernel_size=3, padding=1, bias=False)
    self.relu1 = nn.ReLU(inplace=True)
    # hidden layers
    hidden_layers = []
    for i in range(18):
      hidden_layers.append(nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, padding=1, bias=False))
      hidden_layers.append(nn.BatchNorm2d(64))
      hidden_layers.append(nn.ReLU(inplace=True))
    self.mid_layer = nn.Sequential(*hidden_layers)
    # out layer
    self.conv3 = nn.Conv2d(in_channels=64, out_channels=3, kernel_size=3, padding=1, bias=False)

  def forward(self, x):
    out = self.relu1(self.conv1(x))
    out = self.mid_layer(out)
    out = self.conv3(out)
    return out

In [ ]:
import torchvision

# Create our initial model (WILL BE LOADED IN LATER)
denoising_cnn_official=DenoiseCNN().to(device)

# Create loss fn and move it to the device.
criterion_dcnn=nn.MSELoss()
criterion_dcnn=criterion_dcnn.to(device)

# Create the optimizer
lr=1e-4
optimizer_dcnn=optim.RMSprop(denoising_cnn_official.parameters(), lr=lr)

In [ ]:
# Function for adding photon noise(which is what you would expect in a CT or MRI Scanner)
class AddPoissonNoise(object):
    def __init__(self, mean=0., std=.2):
        self.std = std
        self.mean = mean

    def __call__(self, tensor):
        tensor=tensor.to(device)
        # For example, scale to 255 if the image is in [0, 1] range.
        scaled_image = tensor * 255

        # Convert to integer for Poisson sampling (photon counts)
        scaled_image_int = torch.round(scaled_image).long()

        # Generate Poisson noise based on the intensity (scaled_image_int)
        noisy_image_int = torch.poisson(abs(scaled_image_int.float()))

        # Scale the noisy image back to [0, 1]
        noisy_image = noisy_image_int.float() / 255

        # Ensure the noisy image is in the valid range [0, 1]
        noisy_image = torch.clamp(noisy_image, 0.0, 1.0).to(device)

        return noisy_image

    def __repr__(self):
        return self.__class__.__name__ + '(mean={0}, std={1})'.format(self.mean, self.std)

In [ ]:
#training loop
def train_dcnn(model, train_loader, criterion, optimizer, epochs_dcnn=15):
  model.train()
  # Define first loop
  for epoch in range(epochs_dcnn):
    running_loss=0.0
    # Define Second Loop
    for images, _ in train_loader:
      # Move data to device
      images = images.to(device)
      #Set optimizer to zero grad and make noisy image
      optimizer.zero_grad()
      noise_image=AddPoissonNoise().__call__(images).to(device)
      # Forward Pass
      output=model(noise_image)
      loss=criterion(output, images)

      # Backward Pass and opt
      loss.backward()
      optimizer.step()

      # Get running loss
      running_loss+=loss.item()

      # Save model
      torch.save(model, "dcnn.pth")
    # Print loss
    epoch_loss = running_loss / len(train_loader)
    print(f"Epoch {epoch+1}/{epochs_dcnn} - Loss: {epoch_loss:.4f}")

In [ ]:
# Import math modeule for psnr
from math import log10

In [ ]:
# Evaluate the DCNN and show PSNR if possible :|

def test_dcnn(model, test_loader, criterion):
  avg_psnr=0.0
  model.eval()
  with torch.no_grad():
    for images, _ in test_loader:
      # Set up target and noisy image
      target=images
      noise_target=AddPoissonNoise().__call__(images)

      target=target.to(device)
      noise_target=noise_target.to(device)

      output=model(noise_target)
      mse=criterion(output, target)
      psnr= 10 * log10(1 / mse.item())
  print("===> Avg. PSNR with 10 * log10(1/mse.item()): {:.4f} dB".format(avg_psnr))

# !!! LOAD IN THE MODEL WHEN NEEDED !!!

In [ ]:
# Here, you can load in the model from the pth file provided...right?

denoising_cnn_official=torch.load("denoising_cnn_official.pth", weights_only=False, map_location=device)

## ViM PORTION

# Combine

In [ ]:
import torch
import torch.nn as nn

class CombinedVDCNN(nn.Module):
  def __init__(self, model):
    super().__init__()
    self.dcnn=denoising_cnn_official
    self.model=model
  def forward(self, x):
    x=self.dcnn(x)
    x=self.model(x)
    return x

combvitdcnn=CombinedVDCNN(new_vit_pneumnist)
combvimdcnn=CombinedVDCNN(new_vim_pneumnist)

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

# Define the model with multiple transformer blocks
class ComplexModel(nn.Module):
    def __init__(self):
        super(ComplexModel, self).__init__()

        # Define the initial fully connected layers
        self.fc1 = nn.Linear(2, 64)   # First layer: 2 inputs -> 64 neurons
        self.fc2 = nn.Linear(64, 128) # Second layer: 64 neurons -> 128 neurons

        # Transformer block (with more layers)
        self.transformer_layer = nn.TransformerEncoderLayer(d_model=128, nhead=4)
        self.transformer_encoder = nn.TransformerEncoder(self.transformer_layer, num_layers=1)  # You can keep this as 1 layer

        # Additional fully connected layers after transformer
        self.fc3 = nn.Linear(128, 64) # Third layer: 128 neurons -> 64 neurons
        self.fc4 = nn.Linear(64, 1)   # Output layer: 64 neurons -> 1 output

        # Activation functions
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()   # Sigmoid activation for the output

    def forward(self, x):
        # Pass input through each layer before transformer
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))

        # Transformer expects input in (seq_len, batch_size, model_dim) format
        # Add an extra dimension for sequence length (seq_len = 1000)
        batch_size = x.size(0)  # Get the batch size
        seq_len = 1000  # Set sequence length to 1000

        # Reshape the input tensor to (seq_len, batch_size, model_dim)
        x = x.unsqueeze(0).expand(seq_len, batch_size, -1)  # Expand to (1000, batch_size, 128)

        # Pass through transformer encoder
        x = self.transformer_encoder(x)

        # Remove the sequence length dimension
        x = x.mean(dim=0)  # Average across the sequence length dimension

        # Pass through the remaining fully connected layers
        x = self.relu(self.fc3(x))
        x = self.fc4(x)  # No activation here for final output

        return x

In [ ]:
import torch
import torch.nn as nn

# Define the Gated MLP Class with Sigmoid for gating
class GatedMLPHead(nn.Module):
    def __init__(self, input_dim):
        super(GatedMLPHead, self).__init__()

        # The gating mechanism: learnable parameters to gate the inputs
        self.gate = nn.Sequential(
            nn.Linear(input_dim, 1),  # Single gate for each input (ViT, ViM)
            nn.Sigmoid()  # Sigmoid to get weights between 0 and 1
        )

        # The shared MLP layers after gating
        self.mlp = nn.Sequential(
            nn.Linear(input_dim, 224),
            nn.ReLU(),
            nn.Linear(224, 64),
            nn.ReLU(),
            nn.Linear(64, 16),
            nn.ReLU(),
            nn.Linear(16, 2),
            nn.GELU(),
            nn.Linear(2, 1),
            nn.Softmax(dim=1)  # Output probabilities
        )

    def forward(self, combined_input):
        # Apply the gating mechanism
        gate_weights = self.gate(combined_input)  # (batch_size, 1)

        # Perform the gated combination (element-wise multiplication)
        gated_input = gate_weights * combined_input

        # Pass through the MLP
        output = self.mlp(gated_input)
        return output

# Create the combination of ViMamba and the ViT
class CombinedViTM(nn.Module):
    def __init__(self, vit, vim):
        super(CombinedViTM, self).__init__()

        # Establish our ViMamba and ViT INSIDE the combined model
        self.vit = vit
        self.vim = vim

        # Use the Gated MLP Head instead of the original MLP
        self.head = GatedMLPHead(input_dim=2)

    def forward(self, x):
        # Process the results from both the ViMamba and the ViT
        vit_result = self.vit(x)  # Assume output shape (batch_size, 1)
        vim_result = self.vim(x)  # Assume output shape (batch_size, 1)

        # Concatenate the results from ViT and ViMamba
        combined_result = torch.cat((vit_result, vim_result), dim=1)

        # Process through the gated MLP head
        result = self.head(combined_result)

        return result

In [ ]:
# Create the final model, with the Denoising CNN and merged ViT and ViMamba complete!

class done(nn.Module):
  def __init__(self, dcnn, combined):
    super(done, self).__init__()
    # Initialize the models
    self.dcnn=dcnn
    self.combined=combined

  def forward(self, x):
    # Simple enough!
    x=self.dcnn(x)
    x=self.combined(x)
    return x

# Theoretical is done...

## IMPORT IN THE Comb!

In [ ]:
# Combine
combined_pneumnist=CombinedViTM(new_vit_pneumnist, new_vim_pneumnist)

In [ ]:
# Done
done_pneumnist=done(denoising_cnn_official, combined_pneumnist)
done_pneumnist=done_pneumnist.to(device)

In [ ]:
sum(p.numel() for p in done_pneumnist.parameters() if p.requires_grad)

59934559

In [ ]:
# Create loss function and optimizer
loss_fn=torch.nn.BCEWithLogitsLoss()
loss_fn_alt=torch.nn.BCELoss()

In [ ]:
# Train function
def train_singular(model, train_loader, criterion, optimizer,  loss_list, acc_list, path="", epochs=125):
  for epoch in range(epochs):
    model.train()  # Set model to training mode
    running_loss = 0.0
    correct = 0
    total = 0

    for images, labels in train_loader:
        optimizer.zero_grad()
        images, labels=images.to(device), labels.to(device)
        # Forward pass
        outputs = model(images)
        labels = labels.float().view(-1, 1)  # Reshape labels to match output
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        # Track accuracy
        running_loss += loss.item()
        predicted = (outputs > 0.5).float()  # Convert sigmoid output to binary prediction
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    epoch_loss = running_loss / len(train_loader)
    epoch_accuracy = 100 * correct / total

    print(f"Epoch [{epoch+1}/{epochs}], Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.20f}%")
    loss_list.append(epoch_loss)
    acc_list.append(epoch_accuracy)

In [ ]:
# Train function
def train_combined(model, train_loader, criterion, optimizer, epochs=100):
  for epoch in range(epochs):
    model.train()  # Set model to training mode
    running_loss = 0.0
    correct = 0
    total = 0

    for images, labels in train_loader:
        optimizer.zero_grad()
        images, labels=images.to(device), labels.to(device)
        # Forward pass
        outputs = model(images).float()
        loss = criterion(outputs, labels.float())

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        # Track accuracy
        running_loss += loss.item()
        predicted = (outputs > 0.5).float()  # Convert sigmoid output to binary prediction
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    epoch_loss = running_loss / len(train_loader)
    epoch_accuracy = 100 * correct / total
    print(f"Epoch [{epoch+1}/{epochs}], Loss: {epoch_loss:.25f}, Accuracy: {epoch_accuracy:.2f}%")

In [ ]:
import numpy as np

done_pneumnist_predicted=torch.Tensor([])
real=torch.Tensor([])

def evaluate(model, dataloader, loss_fn, real, done_pneumnist_predicted):
  # Evaluate the model
  model.eval()  # Set model to evaluation mode
  correct = 0
  total = 0
  with torch.no_grad():
      for images, labels in dataloader:
          images, labels=images.to(device),labels.to(device)
          outputs = model(images)
          labels = labels.float().view(-1, 1)
          real=real.to(device)
          done_pneumnist_predicted=done_pneumnist_predicted.to(device)
          real=torch.cat((real, labels), dim=0).cuda()
          predicted = (outputs > 0.5).float()  # Convert sigmoid output to binary prediction
          done_pneumnist_predicted=torch.cat((done_pneumnist_predicted, predicted), dim=0).cuda()
          total += labels.size(0)
          correct += (predicted == labels).sum().item()

  accuracy = 100 * correct / total
  print(f"Test Accuracy: {accuracy:.2f}%")
  print(f"real: {real}, predicted={done_pneumnist_predicted}")

In [ ]:
clear_gpu()

In [ ]:
!export PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True

In [ ]:
optimizer_vit=torch.optim.Adadelta(new_vit_pneumnist.parameters(), lr=0.001)

In [ ]:
optimizer_vim=torch.optim.Adadelta(new_vim_pneumnist.parameters(), lr=0.003)

# LET'S USE LIGHTNING

In [ ]:
import torch
import torch.nn as nn

def train_binary_classification(model, train_loader, criterion, optimizer, epochs=10):
    for epoch in range(epochs):
        model.train()  # Set model to training mode
        running_train_loss = 0.0
        correct_train = 0
        total_train = 0

        # Training loop
        for inputs, labels in train_loader:
            # Forward pass
            outputs = model(inputs)
            inputs, labels = inputs.to(device), labels.to(device)
            # Calculate loss
            loss = criterion(outputs.squeeze(), labels.float())  # Squeeze outputs to match labels
            running_train_loss += loss.item()

            # Zero the parameter gradients
            optimizer.zero_grad()

            # Backward pass and optimize
            loss.backward()
            optimizer.step()

            # Calculate accuracy
            predicted = (torch.sigmoid(outputs) > 0.5).float()
            correct_train += (predicted == labels).sum().item()
            total_train += labels.size(0)

        # Calculate average training loss and accuracy for the epoch
        avg_train_loss = running_train_loss / len(train_loader)
        train_accuracy = correct_train / total_train

        # Print epoch results
        print(f"Epoch {epoch + 1}/{epochs}:")
        print(f"  Train Loss: {avg_train_loss:.4f}, Train Accuracy: {train_accuracy:.4f}")

    print("Training complete.")


In [ ]:
combvitdcnn_opt=torch.optim.Adadelta(combvitdcnn.parameters(), lr=0.01)
combvimdcnn_opt=torch.optim.Adadelta(combvimdcnn.parameters(), lr=0.01)

In [ ]:
vit_loss_list=[]
vit_acc_list=[]

In [ ]:
clear_gpu()
vim_loss_list=[]
vim_acc_list=[]

In [ ]:
torch.cuda.empty_cache()

In [ ]:
head = ComplexModel()
head = head.to(device)
optimizer_head = torch.optim.Adadelta(head.parameters(), lr=0.01)

import torch
import torch.nn as nn

class CustomModel(nn.Module):
    def __init__(self, dcnn, vit, vim, head):
        super(CustomModel, self).__init__()

        # Assign each module to instance variables
        self.dcnn = dcnn

        # Convolution layers to increase channels from 3 to 64
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1)

        # Additional convolutions to bring channels back to 3
        self.conv4 = nn.Conv2d(in_channels=64, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.conv5 = nn.Conv2d(in_channels=32, out_channels=16, kernel_size=3, stride=1, padding=1)
        self.conv6 = nn.Conv2d(in_channels=16, out_channels=3, kernel_size=3, stride=1, padding=1)

        self.vit = vit
        self.vim = vim
        self.head = head

    def forward(self, x):
        # Forward pass through dcnn
        x = self.dcnn(x)
        torch.cuda.empty_cache()
        # Apply convolutions to transform channels from 3 -> 64 -> 3
        x = self.conv1(x)  # 3 -> 16
        torch.cuda.empty_cache()
        x = self.conv2(x)  # 16 -> 32
        torch.cuda.empty_cache()
        x = self.conv3(x)  # 32 -> 64
        torch.cuda.empty_cache()
        x = self.conv4(x)  # 64 -> 32
        torch.cuda.empty_cache()
        x = self.conv5(x)  # 32 -> 16
        torch.cuda.empty_cache()
        x = self.conv6(x)  # 16 -> 3
        torch.cuda.empty_cache()

        # Forward pass through vit and vim
        vit_out = self.vit(x)
        torch.cuda.empty_cache()
        vim_out = self.vim(x)
        torch.cuda.empty_cache()

        # Concatenate the outputs of vit and vim along the feature dimension (dim=1)
        x = torch.cat((vit_out, vim_out), dim=1)

        # Pass the concatenated output to the head
        x = self.head(x)
        return x

c=CustomModel(denoising_cnn_official, new_vit_pneumnist, new_vim_pneumnist, head)
c.load_state_dict(torch.load("c_sd.pth", weights_only=True), strict=False)
c=c.to(device)
optimizer_c=torch.optim.Adadelta(c.parameters(), lr=0.01)
vit_loss_list=[]
vit_acc_list=[]
train_singular(c, train_pneumnist, loss_fn, optimizer_c, vit_loss_list, vit_acc_list, epochs=14)

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/transformer.py:379: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(


In [ ]:
evaluate(c, test_pneumnist_dataloader, loss_fn, real, done_pneumnist_predicted)

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np
from torchvision import transforms
from PIL import Image


class NpzDataset(Dataset):
    def __init__(self, npz_file_path, img, label, transform=None,):
        # Load the .npz file
        data = np.load(npz_file_path)

        # Assuming the .npz file contains arrays named 'images' and 'labels'
        self.images = data['train_images']
        self.labels = data['train_labels']

        # Set up the transform
        self.transform = transform

    def __len__(self):
        # Return the number of samples
        return len(self.images)

    def __getitem__(self, idx):
        # Get an image and label
        image = self.images[idx]
        label = self.labels[idx]

        # Convert image from NumPy array to PIL image (required for torchvision transforms)
        image = Image.fromarray(image.astype('uint8'))  # Convert to uint8 for compatibility

        # Apply transform if any
        if self.transform:
            image = self.transform(image)

        # Convert label to tensor (for classification task, long for labels)
        label = torch.tensor(label, dtype=torch.long)

        return image, label

# Define the transformations
transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=3),  # Convert grayscale to 3 channels
    transforms.ToTensor(),  # Convert image to a PyTorch tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  # Normalize
])

# Initialize the dataset with transformations
npz_file_path = '/content/breastmnist_224.npz'  # Path to your .npz file
dataset_train = NpzDataset(npz_file_path, "train_images", "train_labels", transform=transform)
dataset_test = NpzDataset(npz_file_path, "test_images", "test_labels", transform=transform)

# Create a DataLoader
breastmnist_train = DataLoader(dataset_train, batch_size=32, shuffle=True)
breastmnist_test = DataLoader(dataset_test, batch_size=32, shuffle=False)

train_singular(c, breastmnist_train, loss_fn, optimizer_c, vit_loss_list, vit_acc_list, epochs=14)

In [ ]:
torch.cuda.empty_cache()
evaluate(c, breastmnist_test, loss_fn, real, done_pneumnist_predicted)

In [ ]:
#torch.save(c.state_dict(), "c_sd.pth")

In [ ]:
sum(p.numel() for p in done_pneumnist.parameters() if p.requires_grad)+sum(p.numel() for p in head.parameters())

# TEST OUT OTHER MODELS

In [ ]:
def count_params(model):
  print(sum(p.numel() for p in model.parameters()))

In [ ]:
import torch
import torchvision.models as models

# Load the pre-trained ResNet-18 model
model = models.resnet18(pretrained=True)

# If you want to freeze all the layers except the final fully connected layer:
for param in model.parameters():
    param.requires_grad = False

# Modify the final fully connected layer to match your number of classes
num_ftrs = model.fc.in_features
model.fc = torch.nn.Linear(num_ftrs, 1) # Replace 10 with your number of classes

# Move the model to the desired device (CPU or GPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [ ]:
count_params(model)

In [ ]:
evaluate(model, test_pneumnist_dataloader, loss_fn, real, done_pneumnist_predicted)

In [ ]:
import torch
import torchvision.models as models

# Load the pre-trained ResNet-18 model
model = models.resnet34(pretrained=True)

# If you want to freeze all the layers except the final fully connected layer:
for param in model.parameters():
    param.requires_grad = False

# Modify the final fully connected layer to match your number of classes
num_ftrs = model.fc.in_features
model.fc = torch.nn.Linear(num_ftrs, 1) # Replace 10 with your number of classes

# Move the model to the desired device (CPU or GPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
count_params(model)

evaluate(model, test_pneumnist_dataloader, loss_fn, real, done_pneumnist_predicted)

In [ ]:
import torch
import torchvision.models as models

# Load the pre-trained ResNet-18 model
model = models.resnet50(pretrained=True)

# If you want to freeze all the layers except the final fully connected layer:
for param in model.parameters():
    param.requires_grad = False

# Modify the final fully connected layer to match your number of classes
num_ftrs = model.fc.in_features
model.fc = torch.nn.Linear(num_ftrs, 1) # Replace 10 with your number of classes

# Move the model to the desired device (CPU or GPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
count_params(model)

evaluate(model, test_pneumnist_dataloader, loss_fn, real, done_pneumnist_predicted)

In [ ]:
import torch
import torchvision.models as models

# Load the pre-trained ResNet-18 model
model = models.resnet101(pretrained=True)

# If you want to freeze all the layers except the final fully connected layer:
for param in model.parameters():
    param.requires_grad = False

# Modify the final fully connected layer to match your number of classes
num_ftrs = model.fc.in_features
model.fc = torch.nn.Linear(num_ftrs, 1) # Replace 10 with your number of classes

# Move the model to the desired device (CPU or GPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
count_params(model)

evaluate(model, test_pneumnist_dataloader, loss_fn, real, done_pneumnist_predicted)

In [ ]:
import torch
import torchvision.models as models

# Load the pre-trained ResNet-18 model
model = models.resnet152(pretrained=True)

# If you want to freeze all the layers except the final fully connected layer:
for param in model.parameters():
    param.requires_grad = False

# Modify the final fully connected layer to match your number of classes
num_ftrs = model.fc.in_features
model.fc = torch.nn.Linear(num_ftrs, 1) # Replace 10 with your number of classes

# Move the model to the desired device (CPU or GPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
count_params(model)

evaluate(model, test_pneumnist_dataloader, loss_fn, real, done_pneumnist_predicted)

In [ ]:
import torch
import torchvision.models as models

# Load the pre-trained ResNet-18 model
model = models.vgg16(pretrained=True)

# If you want to freeze all the layers except the final fully connected layer:
for param in model.parameters():
    param.requires_grad = False

# Modify the final fully connected layer to match your number of classes
model.classifier[6] = torch.nn.Linear(4096, 1) # Replace 10 with your number of classes

# Move the model to the desired device (CPU or GPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
count_params(model)

evaluate(model, test_pneumnist_dataloader, loss_fn, real, done_pneumnist_predicted)

In [ ]:
import torch
import torchvision.models as models

# Load the pre-trained ResNet-18 model
model = models.mobilenet_v3_large(pretrained=True)

model
# If you want to freeze all the layers except the final fully connected layer:
for param in model.parameters():
    param.requires_grad = False

# Modify the final fully connected layer to match your number of classes
model.classifier[3] = torch.nn.Linear(1280, 1) # Replace 10 with your number of classes

# Move the model to the desired device (CPU or GPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
count_params(model)

evaluate(model, test_pneumnist_dataloader, loss_fn, real, done_pneumnist_predicted)

In [ ]:
import torch
import torchvision.models as models

# Load the pre-trained ResNet-18 model
model = models.densenet201(pretrained=True)

model
# If you want to freeze all the layers except the final fully connected layer:
for param in model.parameters():
    param.requires_grad = False

# Modify the final fully connected layer to match your number of classes
model.classifier = torch.nn.Linear(1920, 1) # Replace 10 with your number of classes

# Move the model to the desired device (CPU or GPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
count_params(model)

evaluate(model, test_pneumnist_dataloader, loss_fn, real, done_pneumnist_predicted)

# TEST OUT OTHER MODELS BCANCER

In [ ]:
def count_params(model):
  print(sum(p.numel() for p in model.parameters()))

In [ ]:
import torch
import torchvision.models as models

# Load the pre-trained ResNet-18 model
model = models.resnet18(pretrained=True)

# If you want to freeze all the layers except the final fully connected layer:
for param in model.parameters():
    param.requires_grad = False

# Modify the final fully connected layer to match your number of classes
num_ftrs = model.fc.in_features
model.fc = torch.nn.Linear(num_ftrs, 1) # Replace 10 with your number of classes

# Move the model to the desired device (CPU or GPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [ ]:
count_params(model)

In [ ]:
evaluate(model, breastmnist_test, loss_fn, real, done_pneumnist_predicted)

In [ ]:
import torch
import torchvision.models as models

# Load the pre-trained ResNet-18 model
model = models.resnet34(pretrained=True)

# If you want to freeze all the layers except the final fully connected layer:
for param in model.parameters():
    param.requires_grad = False

# Modify the final fully connected layer to match your number of classes
num_ftrs = model.fc.in_features
model.fc = torch.nn.Linear(num_ftrs, 1) # Replace 10 with your number of classes

# Move the model to the desired device (CPU or GPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
count_params(model)

evaluate(model, breastmnist_test, loss_fn, real, done_pneumnist_predicted)

In [ ]:
import torch
import torchvision.models as models

# Load the pre-trained ResNet-18 model
model = models.resnet50(pretrained=True)

# If you want to freeze all the layers except the final fully connected layer:
for param in model.parameters():
    param.requires_grad = False

# Modify the final fully connected layer to match your number of classes
num_ftrs = model.fc.in_features
model.fc = torch.nn.Linear(num_ftrs, 1) # Replace 10 with your number of classes

# Move the model to the desired device (CPU or GPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
count_params(model)

evaluate(model, breastmnist_test, loss_fn, real, done_pneumnist_predicted)

In [ ]:
import torch
import torchvision.models as models

# Load the pre-trained ResNet-18 model
model = models.resnet101(pretrained=True)

# If you want to freeze all the layers except the final fully connected layer:
for param in model.parameters():
    param.requires_grad = False

# Modify the final fully connected layer to match your number of classes
num_ftrs = model.fc.in_features
model.fc = torch.nn.Linear(num_ftrs, 1) # Replace 10 with your number of classes

# Move the model to the desired device (CPU or GPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
count_params(model)

evaluate(model, breastmnist_test, loss_fn, real, done_pneumnist_predicted)

In [ ]:
import torch
import torchvision.models as models

# Load the pre-trained ResNet-18 model
model = models.resnet152(pretrained=True)

# If you want to freeze all the layers except the final fully connected layer:
for param in model.parameters():
    param.requires_grad = False

# Modify the final fully connected layer to match your number of classes
num_ftrs = model.fc.in_features
model.fc = torch.nn.Linear(num_ftrs, 1) # Replace 10 with your number of classes

# Move the model to the desired device (CPU or GPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
count_params(model)

evaluate(model, breastmnist_test, loss_fn, real, done_pneumnist_predicted)

In [ ]:
import torch
import torchvision.models as models

# Load the pre-trained ResNet-18 model
model = models.vgg16(pretrained=True)

# If you want to freeze all the layers except the final fully connected layer:
for param in model.parameters():
    param.requires_grad = False

# Modify the final fully connected layer to match your number of classes
model.classifier[6] = torch.nn.Linear(4096, 1) # Replace 10 with your number of classes

# Move the model to the desired device (CPU or GPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
count_params(model)

evaluate(model, breastmnist_test, loss_fn, real, done_pneumnist_predicted)

In [ ]:
import torch
import torchvision.models as models

# Load the pre-trained ResNet-18 model
model = models.mobilenet_v3_large(pretrained=True)

model
# If you want to freeze all the layers except the final fully connected layer:
for param in model.parameters():
    param.requires_grad = False

# Modify the final fully connected layer to match your number of classes
model.classifier[3] = torch.nn.Linear(1280, 1) # Replace 10 with your number of classes

# Move the model to the desired device (CPU or GPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
count_params(model)

evaluate(model, breastmnist_test, loss_fn, real, done_pneumnist_predicted)

In [ ]:
import torch
import torchvision.models as models

# Load the pre-trained ResNet-18 model
model = models.densenet201(pretrained=True)

model
# If you want to freeze all the layers except the final fully connected layer:
for param in model.parameters():
    param.requires_grad = False

# Modify the final fully connected layer to match your number of classes
model.classifier = torch.nn.Linear(1920, 1) # Replace 10 with your number of classes

# Move the model to the desired device (CPU or GPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
count_params(model)

evaluate(model, breastmnist_test, loss_fn, real, done_pneumnist_predicted)

In [ ]:
c_new=CustomModel(denoising_cnn_official, new_vit_pneumnist, new_vim_pneumnist, head)
c_new.load_state_dict(torch.load("c_sd.pth", weights_only=True))